In [ ]:
# default_exp simulator

# Simulator

> Defines unified interface to generic simulators.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# hide
# export
from fastcore.basics import patch

In [ ]:
#export
class Simulator:
    """Interface for quantum state simulation"""
    def __init__(self, num_qubits):
        self.n_qubits = num_qubits

In [ ]:
#export
@patch
def say(self:Simulator, hi):
    return f'hi {hi}'

In [ ]:
s = Simulator(4)
s.say('jo')

'hi jo'